# 🎓 UAS FRAUD DETECTION - LOCAL EXECUTION (RTX 2060)

**Author:** BTC Goku ID  
**Date:** December 30, 2025  
**Hardware:** RTX 2060 6GB  
**Environment:** Local PC - VSCode  

---

## 📋 Setup Instructions:

### 1. Install Libraries (Run in terminal):
```bash
pip install pandas polars numpy matplotlib seaborn scikit-learn
pip install imbalanced-learn tqdm jupyter ipykernel
pip install lightgbm xgboost catboost optuna
pip install tensorflow --upgrade  # GPU support
pip install scikeras
```

### 2. Folder Structure:
```
C:\Users\hamda\OneDrive\Documents\SEMESTER 7\MACHINE LEARNING\
├── transaction\              # Raw data
│   ├── train_transaction.csv
│   └── test_transaction.csv
├── Fraud_Detection_LOCAL_FIXED.ipynb  # This notebook
└── outputs\                  # Generated files
```

### 3. Update BASE_PATH in Cell 3 if needed!

---

In [4]:
import sys
print(f"Python: {sys.executable}")

# Test imports
import pandas as pd
import polars as pl
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import optuna

print("\n✅ ALL LIBRARIES LOADED!")
print(f"LightGBM: {lgb.__version__}")
print(f"XGBoost: {xgb.__version__}")

Python: c:\Users\hamda\.conda\envs\fraud_gpu\python.exe

✅ ALL LIBRARIES LOADED!
LightGBM: 4.6.0
XGBoost: 3.1.2


In [5]:
"""
📚 BLOK 2: IMPORT LIBRARIES & GPU SETUP
═══════════════════════════════════════════════════════
Run time: ~10 seconds
Description: Import all required libraries and configure GPU
"""

# Core libraries
import sys
import os
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
from tqdm import tqdm
import pickle
import json
from datetime import datetime

# ML Libraries
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.metrics import (
    roc_auc_score, average_precision_score, 
    confusion_matrix, classification_report,
    precision_recall_curve, roc_curve, f1_score,
    precision_score, recall_score
)

# Imbalance Handling
from imblearn.over_sampling import SMOTE

# ML Models
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

# Hyperparameter Optimization
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Seeds for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

print("="*70)
print("🚀 UAS FRAUD DETECTION - LOCAL EXECUTION")
print("="*70)
print(f"Python version: {sys.version.split()[0]}")
print(f"Working directory: {os.getcwd()}")
print(f"Pandas: {pd.__version__}")
print(f"Polars: {pl.__version__}")
print(f"TensorFlow: {tf.__version__}")
print(f"LightGBM: {lgb.__version__}")
print(f"XGBoost: {xgb.__version__}")
print("="*70)

# ================================================================
# GPU CONFIGURATION
# ================================================================
print("\n" + "="*70)
print("🎮 GPU DETECTION & CONFIGURATION")
print("="*70)

# Check TensorFlow GPU
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # ✅ CRITICAL: Enable memory growth for RTX 2060 6GB
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print(f"✅ GPU Detected: {len(gpus)} device(s)")
        for i, gpu in enumerate(gpus):
            details = tf.config.experimental.get_device_details(gpu)
            print(f"   GPU {i}: {details.get('device_name', 'RTX 2060')}")
        
        print("   Memory growth: ENABLED")
        print("   ✓ XGBoost: Will use GPU (gpu_hist)")
        print("   ✓ CatBoost: Will use GPU (task_type='GPU')")
        print("   ✓ LightGBM: Will use CPU (no GPU support on Windows)")
        
    except RuntimeError as e:
        print(f"❌ GPU configuration error: {e}")
else:
    print("⚠️ No GPU detected - all models will use CPU")
    print("   Check:")
    print("   1. NVIDIA drivers installed? Run: nvidia-smi")
    print("   2. TensorFlow with CUDA? Run: pip install tensorflow[and-cuda]")

print("="*70)
print("\n✅ ALL LIBRARIES LOADED & GPU CONFIGURED!")

🚀 UAS FRAUD DETECTION - LOCAL EXECUTION
Python version: 3.10.19
Working directory: c:\Users\hamda\OneDrive\Documents\SEMESTER 7\MACHINE LEARNING\fraud
Pandas: 2.3.3
Polars: 1.36.1
TensorFlow: 2.16.1
LightGBM: 4.6.0
XGBoost: 3.1.2

🎮 GPU DETECTION & CONFIGURATION
⚠️ No GPU detected - all models will use CPU
   Check:
   1. NVIDIA drivers installed? Run: nvidia-smi
   2. TensorFlow with CUDA? Run: pip install tensorflow[and-cuda]

✅ ALL LIBRARIES LOADED & GPU CONFIGURED!


In [7]:
"""
📂 BLOK 3: SET PATHS & LOAD DATA
═══════════════════════════════════════════════════════
Run time: ~1-2 minutes
Description: Set file paths and load datasets
"""

# ✅ FIX: Use raw string to avoid escape character issues
BASE_PATH = r"C:\Users\hamda\OneDrive\Documents\SEMESTER 7\MACHINE LEARNING\fraud\transaction"

# Verify path exists
if not os.path.exists(BASE_PATH):
    print(f"❌ ERROR: Path not found!")
    print(f"   Looking for: {BASE_PATH}")
    print(f"\n   Please update BASE_PATH above to match your actual folder location")
    raise FileNotFoundError(f"Path not found: {BASE_PATH}")

print("\n" + "="*70)
print("📂 LOADING DATASETS...")
print("="*70)
print(f"Base path: {BASE_PATH}")

# Load with Polars (memory efficient)
print("\n1. Loading train_transaction.csv...")
train_transaction = pl.read_csv(f"{BASE_PATH}/train_transaction.csv")
print(f"   ✓ Train shape: {train_transaction.shape}")
print(f"   ✓ Memory: {train_transaction.estimated_size('mb'):.2f} MB")

print("\n2. Loading test_transaction.csv...")
test_transaction = pl.read_csv(f"{BASE_PATH}/test_transaction.csv")
print(f"   ✓ Test shape: {test_transaction.shape}")
print(f"   ✓ Memory: {test_transaction.estimated_size('mb'):.2f} MB")

print("\n" + "="*70)
print("✅ DATA LOADED SUCCESSFULLY!")
print("="*70)

# Preview
print("\n📊 TRAIN DATA PREVIEW:")
display(train_transaction.head().to_pandas())

print("\n📊 BASIC INFO:")
print(f"   Training samples: {len(train_transaction):,}")
print(f"   Test samples: {len(test_transaction):,}")
print(f"   Features: {train_transaction.shape[1]}")

# Check for target variable
if 'isFraud' in train_transaction.columns:
    fraud_count = train_transaction['isFraud'].sum()
    fraud_pct = (fraud_count / len(train_transaction)) * 100
    print(f"\n📊 TARGET DISTRIBUTION:")
    print(f"   Fraud: {fraud_count:,} ({fraud_pct:.2f}%)")
    print(f"   Not Fraud: {len(train_transaction) - fraud_count:,} ({100-fraud_pct:.2f}%)")
    print(f"   ⚠️ Class imbalance: {(len(train_transaction) - fraud_count) / fraud_count:.1f}:1")


📂 LOADING DATASETS...
Base path: C:\Users\hamda\OneDrive\Documents\SEMESTER 7\MACHINE LEARNING\fraud\transaction

1. Loading train_transaction.csv...
   ✓ Train shape: (590540, 394)
   ✓ Memory: 1747.16 MB

2. Loading test_transaction.csv...
   ✓ Test shape: (506691, 393)
   ✓ Memory: 1495.48 MB

✅ DATA LOADED SUCCESSFULLY!

📊 TRAIN DATA PREVIEW:


ModuleNotFoundError: No module named 'pyarrow'